In [36]:
# import logging
# logging.getLogger().setLevel(logging.CRITICAL)
# # !pip install pylabel > /dev/null

In [37]:
from pylabel import importer

In [38]:
path_to_annotations = "../data/downsample_data/Annotations/V2F_HY_9148_20201221_111302_E_CH0_Seoul_Sun_Mainroad_Day_97331_BBOX.json"
path_to_images = "../data/downsample_data/Images/V2F_HY_9147_20201102_081931_E_CH0_Seoul_Sun_Mainroad_Day_72590.png"

In [ ]:
#Import the dataset into the pylable schema 
dataset = importer.ImportCoco(path_to_annotations, path_to_images= path_to_images, name="BCCD_coco")
dataset.df.head(5)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,../data/downsample_data/Images/V2F_HY_9147_202...,V2F_HY_9147_20201102_081931_E_CH0_Seoul_Sun_Ma...,,1,1281,720,,,1005.171501,295.725438,...,[],,,,,10,Manhole,,,1


In [40]:
dataset.df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   img_folder         1 non-null      object 
 1   img_filename       1 non-null      object 
 2   img_path           1 non-null      object 
 3   img_id             1 non-null      int64  
 4   img_width          1 non-null      int32  
 5   img_height         1 non-null      int32  
 6   img_depth          1 non-null      object 
 7   ann_segmented      1 non-null      object 
 8   ann_bbox_xmin      1 non-null      float64
 9   ann_bbox_ymin      1 non-null      float64
 10  ann_bbox_xmax      1 non-null      float64
 11  ann_bbox_ymax      1 non-null      float64
 12  ann_bbox_width     1 non-null      float64
 13  ann_bbox_height    1 non-null      float64
 14  ann_area           1 non-null      float64
 15  ann_segmentation   1 non-null      object 
 16  ann_iscrowd        1 non-null 

In [41]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")
print(f"Path to annotations:\n{dataset.path_to_annotations}")

Number of images: 1
Number of classes: 1
Classes:['Manhole']
Class counts:
Manhole    1
Name: cat_name, dtype: int64
Path to annotations:
..\data\downsample_data\Annotations


In [42]:
def convert_bbox_coco2yolo(img_width, img_height, bbox):
    """
    Convert bounding box from COCO  format to YOLO format

    Parameters
    ----------
    img_width : int
        width of image
    img_height : int
        height of image
    bbox : list[int]
        bounding box annotation in COCO format: 
        [top left x position, top left y position, width, height]

    Returns
    -------
    list[float]
        bounding box annotation in YOLO format: 
        [x_center_rel, y_center_rel, width_rel, height_rel]
    """
    
    # YOLO bounding box format: [x_center, y_center, width, height]
    # (float values relative to width and height of image)
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

In [43]:
print(dataset.df.cat_id)
print(dataset.df.img_width)
print(dataset.df.img_height)
print(dataset.df.ann_bbox_xmin)
print(dataset.df.ann_bbox_ymin)
print(dataset.df.ann_bbox_xmax)
print(dataset.df.ann_bbox_ymax)
print(dataset.df.ann_bbox_width)
print(dataset.df.ann_bbox_height)

result = convert_bbox_coco2yolo(dataset.df.img_width, dataset.df.img_height, [dataset.df.ann_bbox_xmin, dataset.df.ann_bbox_ymin, dataset.df.ann_bbox_width, dataset.df.ann_bbox_height])

id
0    10
Name: cat_id, dtype: object
id
0    1281
Name: img_width, dtype: int32
id
0    720
Name: img_height, dtype: int32
id
0    1005.171501
Name: ann_bbox_xmin, dtype: float64
id
0    295.725438
Name: ann_bbox_ymin, dtype: float64
id
0    1186.589101
Name: ann_bbox_xmax, dtype: float64
id
0    327.436644
Name: ann_bbox_ymax, dtype: float64
id
0    181.4176
Name: ann_bbox_width, dtype: float64
id
0    31.711207
Name: ann_bbox_height, dtype: float64


In [44]:
result

[id
 0    0.855488
 dtype: float64,
 id
 0    0.432751
 dtype: float64,
 id
 0    0.141622
 dtype: float64,
 id
 0    0.044043
 dtype: float64]